In [1]:
import pandas as pd
import sweetviz
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_score
import pickle
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score




In [2]:
def generate_model_report(y_actual, y_predicted):
    print("Accuracy = " , accuracy_score(y_actual, y_predicted))
    print("Precision = " ,precision_score(y_actual, y_predicted))
    print("Recall = " ,recall_score(y_actual, y_predicted))
    print("F1 Score = " ,f1_score(y_actual, y_predicted))
    pass

In [3]:
df=pd.read_csv("bank.csv")

In [4]:
my_report=sweetviz.analyze([df,"Default"])

:FEATURES DONE:                    |                     | [  0%]   00:14  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:25  -> (00:00 left)


Creating Associations graph... DONE!


In [ ]:
my_report.show_html('Report.html')

In [18]:
col=['Term','FranchiseCode','UrbanRural','MIS_Status',
     'DisbursementGross','ChgOffPrinGr','GrAppv','SBA_Appv']

In [19]:
df=df[col]

In [20]:
for column in['MIS_Status']:
    df[column].fillna(df[column].mode()[0],inplace=True)

In [21]:
df['code']=pd.factorize(df.MIS_Status)[0]

In [22]:
row_indexes=df[df['FranchiseCode']>1].index
df.loc[row_indexes,'isfranchise']='1'
row_indexes=df[df['FranchiseCode']<=1].index
df.loc[row_indexes,'isfranchise']='0'

In [23]:
df.drop(['MIS_Status','FranchiseCode'],axis=1,inplace=True)

In [24]:
df.isfranchise=df.isfranchise.astype(int)

In [25]:
x=df.iloc[:,df.columns!='code']
y=df.code

In [26]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=\
train_test_split(x,y,test_size=0.3,random_state=10)

In [27]:
currency_col = ['DisbursementGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']
x_train[currency_col] = x_train[currency_col].replace('[\$,]', '', regex= True).astype(float)
x_test[currency_col] = x_test[currency_col].replace('[\$,]', '', regex= True).astype(float)

C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
x_test

,Term,UrbanRural,DisbursementGross,ChgOffPrinGr,GrAppv,SBA_Appv,isfranchise
39514,52,0,100000.0,0.0,100000.0,80000.0,0
38778,83,1,46870.0,0.0,27500.0,13750.0,0
56036,60,2,115000.0,0.0,50000.0,25000.0,0
6412,84,1,25000.0,0.0,25000.0,12500.0,0
89663,15,1,83313.0,20989.0,50000.0,25000.0,0
...,...,...,...,...,...,...,...
605,63,0,71000.0,0.0,71000.0,56800.0,0
122382,12,1,150000.0,75673.0,150000.0,75000.0,0
27289,60,1,87300.0,0.0,87300.0,43650.0,0
48723,3,1,50000.0,0.0,50000.0,42500.0,0


In [34]:
#DecisionTree
modelDS=DecisionTreeClassifier(criterion='entropy',random_state=1,max_depth=18,min_samples_leaf=50)
modelDS.fit(x_train,y_train)


y_train_predict=modelDS.predict(x_train)
y_test_predict=modelDS.predict(x_test)
print("actual",y_test)
print("pred",y_test_predict)


print('Train Accuracy',accuracy_score(y_train,y_train_predict)*100)
print('Test Accuracy',accuracy_score(y_test,y_test_predict)*100)
generate_model_report(y_test,y_test_predict)

actual 39514     0
38778     0
56036     0
6412      0
89663     1
         ..
605       0
122382    0
27289     0
48723     0
50108     1
Name: code, Length: 45000, dtype: int64
pred [0 0 0 ... 0 0 1]
Train Accuracy 98.98856179582663
Test Accuracy 99.02
Accuracy =  0.9902
Precision =  0.967456608811749
Recall =  0.9956204379562044
F1 Score =  0.981336493292141


In [16]:
# Saving model to disk
pickle.dump(modelDS, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))

# predict the output
y_pred = model.predict(x_test)
 
# confusion matrix
print('Confusion matrix of Decision model: \n',confusion_matrix(y_test, y_pred),'\n')
 
# show the accuracy
print('Accuracy of Decision mode = ',accuracy_score(y_test, y_pred))

Confusion matrix of Decision model: 
 [[32965   390]
 [   51 11594]] 

Accuracy of Decision mode =  0.9902
